In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [4]:
def Dataloader(data):

    data['Age'].fillna(data['Age'].mean(), inplace=True)
    data['Embarked'].fillna('S', inplace=True) #inplace 써도 되고 함수 지정해도 되고
    data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
    data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

    X = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']] # 고유값 칼럼은 제외하세요 필요한 칼럼만
    return X

X_train = Dataloader(train)
y_train = train['Survived']

X_test = Dataloader(test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # fit_transform이랑 transform 안헷갈리게 구분 잘해서 기억해야됨
X_test = scaler.transform(X_test)


In [5]:
print(X_train.shape, X_test.shape)

(891, 7) (418, 7)


In [6]:
class ANN(nn.Module): # ANN -> 뉴럴 네트워크 / DNN -> Deep 뉴럴 네트워크 (보통 2개 이상의 은닉층을 가지고있는 뉴럴 네트워크를 칭함)
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, 64), # Input값이 왜 7인지 생각 -> 원래는 Intput = 7, Output = 1 따로 설정하거나 매개변수로 만드는게 일반적임
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 1),  # Output은 이진분류모델이니까 1로 설정
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

model = ANN()
model

ANN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=7, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [7]:
loss_fn = nn.BCELoss() # Binary 손실함수
optimizer = optim.Adam(model.parameters(), lr=0.001) # 경사하강법 = SGD

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_train_tensor # 1. 텐서값으로 변환하는 이유 -> 각자 찾아보기 2. When? -> 모델학습 외에 언제 사용되는게 있는지

tensor([[ 0.8274, -0.7377, -0.5925,  ..., -0.4737, -0.5024, -0.5688],
        [-1.5661,  1.3556,  0.6388,  ..., -0.4737,  0.7868,  1.0052],
        [ 0.8274,  1.3556, -0.2847,  ..., -0.4737, -0.4889, -0.5688],
        ...,
        [ 0.8274,  1.3556,  0.0000,  ...,  2.0089, -0.1763, -0.5688],
        [-1.5661, -0.7377, -0.2847,  ..., -0.4737, -0.0444,  1.0052],
        [ 0.8274, -0.7377,  0.1771,  ..., -0.4737, -0.4924,  2.5792]])

In [8]:
def train(model, loss_fn, optimizer, X_train, y_train, epochs=100):
    model.train()
    for epoch in range(epochs):
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")


In [9]:
def test(model, X_test, threshold=0.5):
    model.eval()
    with torch.no_grad():
        test_pred = model(X_test) # y타겟 Survived  0.34549, 0.78153, 0.4845, 0.4866 .... 0, 1
    test_pred = (test_pred >= threshold).float()
    #    test_pred = test_pred.round()
    return test_pred

In [10]:
train(model, loss_fn, optimizer, X_train_tensor, y_train_tensor, epochs=500)
test_pred = test(model, torch.tensor(X_test, dtype=torch.float32))
test_pred

Epoch 0, Loss: 0.6946794986724854
Epoch 10, Loss: 0.5599545836448669
Epoch 20, Loss: 0.4330001473426819
Epoch 30, Loss: 0.4140935242176056
Epoch 40, Loss: 0.39877504110336304
Epoch 50, Loss: 0.388489305973053
Epoch 60, Loss: 0.3794570863246918
Epoch 70, Loss: 0.37067583203315735
Epoch 80, Loss: 0.3617149591445923
Epoch 90, Loss: 0.3518972098827362
Epoch 100, Loss: 0.34097054600715637
Epoch 110, Loss: 0.3289570212364197
Epoch 120, Loss: 0.3164646029472351
Epoch 130, Loss: 0.3052251636981964
Epoch 140, Loss: 0.2949002981185913
Epoch 150, Loss: 0.28562334179878235
Epoch 160, Loss: 0.27992773056030273
Epoch 170, Loss: 0.2703498601913452
Epoch 180, Loss: 0.263126015663147
Epoch 190, Loss: 0.2571137547492981
Epoch 200, Loss: 0.25227296352386475
Epoch 210, Loss: 0.2481759637594223
Epoch 220, Loss: 0.24190030992031097
Epoch 230, Loss: 0.2390567660331726
Epoch 240, Loss: 0.23551087081432343
Epoch 250, Loss: 0.23141342401504517
Epoch 260, Loss: 0.23091799020767212
Epoch 270, Loss: 0.224665388464

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
      